In [1]:
import matplotlib.pyplot as plt
import pandas
import structlog
from astropy import coordinates
from astropy import units as u

from app.lib.storage import postgres


In [ ]:
import os
from dataclasses import dataclass

from experiments import entities, plots


@dataclass
class CrossIDInfo:
    object_id: str
    ra: float
    dec: float
    name: str
    status: entities.CrossIdentificationStatus
    pgc_numbers: list[int]


df = pandas.read_csv("experiments/results/bayes.csv")

df = df[df["status"] == "collision"]
print(df.head())

storage_config = postgres.PgStorageConfig(
    endpoint="dm2.sao.ru", port=5432, dbname="hyperleda", user="hyperleda", password=os.getenv("DB_PASS") or ""
)

rows = df.to_dict(orient="records")

In [ ]:
storage = postgres.PgStorage(storage_config, structlog.get_logger())

storage.connect()

offset = 100
grid_rows = 2
grid_cols = 2
num_objects = grid_rows * grid_cols
selected_objs = [CrossIDInfo(**row) for row in rows[offset:num_objects+offset]]

fig, axes_arr = plt.subplots(grid_rows, grid_cols, figsize=(5 * grid_cols, 5 * grid_rows))

fov = 1 / 60
hips_map = 'CDS/P/DESI-Legacy-Surveys/DR10/color'

axes_flat = axes_arr.flatten()

try:
    all_pgc_numbers = set()
    obj_to_pgcs = []
    for obj in selected_objs:
        pgc_numbers = [int(pgc.strip()) for pgc in str(obj.pgc_numbers).split(",") if pgc.strip()]
        obj_to_pgcs.append(pgc_numbers)
        all_pgc_numbers.update(pgc_numbers)

    all_pgc_numbers = list(all_pgc_numbers)
    all_pgc_data = entities.get_pgc_objects_info(all_pgc_numbers, storage)
    pgc_to_obj = {obj.pgc: obj for obj in all_pgc_data}
except Exception as e:
    raise e
finally:
    storage.disconnect()

for obj, ax, pgc_numbers in zip(selected_objs, axes_flat, obj_to_pgcs, strict=False):
   data = [pgc_to_obj[pgc] for pgc in pgc_numbers if pgc in pgc_to_obj]
   collision_data = [(curr_obj.ra, curr_obj.dec, curr_obj.name) for curr_obj in data]
   coords = coordinates.SkyCoord(ra=obj.ra * u.deg, dec=obj.dec * u.deg)
   print(coords)
   print(collision_data)
   plots.get_hips_map(obj.ra, obj.dec, fov, hips_map, collision_data, ax)


# plt.show()